In [2]:
# Define necessary variables
import ee
import os
import pandas as pd
ee.Initialize()
import numpy as np

classNames = ['Permanent_water', 'Mangrove', 'Herbaceous_wetland', 'Tree_cover', 'Shrubland', 'Sparse', 'Build_environment']
subClassNames = ['Permanent_water', 'Mangrove', 'Dam_tanki', 'Salina', 'Wetland_other', 'Cactus_dominated', 'Deciduous_dominated', 'Shrubland', 
                 'Quarry', 'Degraded_land', 'Sand_coast', 'Rock_coast', 'Rock_formation', 'Dead_mangrove', 'Road', 'Building', 'Sport_field']

classDict = { item: i for i, item in enumerate(classNames)}
subClassDict = { item: i for i, item in enumerate(subClassNames)}
reverse_classDict = {value: key for key, value in classDict.items()}
reverse_subClassDict = {value: key for key, value in subClassDict.items()}
landcoverOffset = {'Herbaceous_Wetland': 2, 'Tree_Cover': 5, 'Sparse': 8, 'Build_Environment': 14}

classesFromSubClasses = {
    'Permanent_water': [0],
    'Mangrove': [1],
    'Herbaceous_wetland': [2,3,4],
    'Tree_cover': [5,6], 
    'Shrubland': [7],
    'Sparse': [8, 9, 10, 11, 12, 13],
    'Build_environment': [14, 15, 16]
}

In [64]:
# Crease .csv from locations in picture metadata
import os
import random
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import csv

def get_exif_data(image_path):
    """Extract and return the EXIF data from an image file."""
    with Image.open(image_path) as img:
        exif_data = img._getexif()
        if exif_data is not None:
            # Decode EXIF data, converting tags from IDs to human-readable names
            exif = {TAGS.get(tag, tag): value for tag, value in exif_data.items()}
            # Extract GPS data if available
            if 'GPSInfo' in exif:
                gps_info = {}
                for key in exif['GPSInfo'].keys():
                    decoded = GPSTAGS.get(key, key)
                    gps_info[decoded] = exif['GPSInfo'][key]
                exif['GPSInfo'] = gps_info
            return exif
        else:
            return {}

def gps_info_to_decimal(gps_info, hemisphere):
    """Convert GPSInfo (degrees, minutes, seconds) to decimal coordinates."""
    degrees, minutes, seconds = gps_info
    decimal = float(degrees) + float(minutes) / 60 + float(seconds) / 3600
    if hemisphere in ['S', 'W']:
        decimal *= -1
    return decimal

image_loc = "./input/pictures"

for folder in os.listdir(image_loc):
    subpath = image_loc + '/' + folder
    output_path = './input/picturesCsv/'+ folder + '.csv'
    
    # This is not yet correct!!
    className = folder.split('.')[0]
    landcover = subClassDict[className]

    with open(output_path, 'w', newline='\n') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['latitude','longitude', 'landcover'])
        for image in os.listdir(subpath):
            image_path = subpath + '/' + image
            exifData = get_exif_data(image_path)
            try: 
                if exifData['GPSInfo']:
                    gps_info = exifData['GPSInfo']
                    
                    # looks wrong but is correct..
                    longitude = gps_info_to_decimal(gps_info['GPSLatitude'], gps_info['GPSLatitudeRef']) 
                    latitude = gps_info_to_decimal(gps_info['GPSLongitude'], gps_info['GPSLongitudeRef']) 
                    row = [latitude, longitude, landcover]
                    csvwriter.writerow(row)
            except: pass

In [68]:
# Crease .csv from Fieldmap observations
import pandas as pd
classification = pd.read_csv("./input/arcGISpoints.csv")
class_counts = classification['Choose Class'].value_counts()

for landcover in classification['Choose Class'].unique():
    try:
        name = reverse_subClassDict.get(int(landcover))
        corresponding_rows = classification[classification['Choose Class'] == landcover]
        csv_data = corresponding_rows[['x', 'y']].copy()
        csv_data['landcover'] = int(landcover)
        csv_data.columns = ['latitude', 'longitude', 'landcover']
        csv_filename = './input/arcGisCsv/' + name + '.csv'

        csv_data.to_csv(csv_filename, index=False, header=['latitude','longitude', 'landcover'])
    except ValueError:
        print('Another Nan')
    except TypeError:
        print('None')

Another Nan


In [ ]:
//  Find Google Earth Engine points and export to assets
//  The following code was executed in the Javascript Google Earth Engine terminal, to make use of the point selection offered through there

//////// Imports
var regVis = {
  min: 0,
  max: 0.6,
  bands: ['B4', 'B3', 'B2'],
};

// Show latest composite to learn points
var landsatImage = ee.Image('projects/aruba-tdf/assets/1-images/2024')

Map.addLayer(landsatImage, regVis, 'landsat')
Map.centerObject(image)
// Select points through map window

// 
// 

// After selecting points, export to featureCollections

var featureCollections = {
  Sea: sea,
  Mangrove: mangroves,
  Dam_Tanki: damsTanki,
  Quarry: quarries,
  Road: road,
  Building: building,
  Sportfield: sportfields,
  Sandcoast: sandShore,
  Degraded_Land: degradedLand,
  Rockformations: rockformations,
  Salina: salina,
  Wetland_Other: wetland,
  Rockcoast: rockCoast,
  Dead_Mangrove: deadMangrove,
  Shrubland: shrubland
};

function exportFeatureCollectionToAsset(collectionName, featureCollection) {
  Export.table.toAsset({
    collection: featureCollection,
    description: collectionName + '_to_asset', // Description of the export
    assetId: 'projects/aruba-tdf/assets/geeReprPoints/' + collectionName, // Set the destination asset ID
  });
}
// Loop over the featureCollections object and export each FeatureCollection
for (var name in featureCollections) {
  exportFeatureCollectionToAsset(name, featureCollections[name]);
}

In [74]:
# Create training and groundTruth dictionary for the subclasses
import random

# csv to EE.Featurecollection function
def inputToEE(file):
    features=[]
    df = pd.read_csv(file)
    for i in range(df.shape[0]):
        x, y, landcover = df.latitude[i],df.longitude[i], df.landcover[i]
        point = ee.Geometry.Point([x, y]) 
        feature = ee.Feature(point, {'landcover': int(landcover)})
        features.append(feature)
    ee_object = ee.FeatureCollection(features)
    
    return ee_object

def popRandomPoints(collection, amt=1):
    """
    Remove amt number of points from collection, 
    return [Removed Points, old Points]
    where both are ee.FeatureCollections([])
    """
    if amt > 0 and amt < collection.size().getInfo(): 
        remaining_points = collection.getInfo()['features'].copy()
        removed_points = random.sample(remaining_points, amt)
        for point in removed_points:
            remaining_points.remove(point)
        return [ee.FeatureCollection(removed_points), ee.FeatureCollection(remaining_points)]
    else:
        return [ee.FeatureCollection([]), collection]

subClassPoints = {key: ee.FeatureCollection([]) for key in subClassDict.keys()}
            
# Load local data
main_folder = './input/'
for folder in ['arcGisCsv', 'picturesCsv']:
    sub_folder = main_folder + folder
    for key in subClassDict.keys():
        if key + '.csv' in os.listdir(sub_folder):
            item_path = sub_folder +'/'+ key + '.csv'
            newCollection = inputToEE(item_path)
            subClassPoints[key] = subClassPoints[key].merge(newCollection) 


# Load non-local data
main_folder = 'projects/aruba-tdf/assets/geeReprPoints'
assets_list = ee.data.listAssets({'parent': main_folder})
asset_ids = [asset['id'] for asset in assets_list['assets']]
for asset in asset_ids:
    className = asset.split('/')[-1]
    newCollection = ee.FeatureCollection(asset)
    subClassPoints[className] = subClassPoints[className].merge(newCollection)

subClassPoints

{'Permanent_water': <ee.featurecollection.FeatureCollection at 0x279a01c0690>,
 'Mangrove': <ee.featurecollection.FeatureCollection at 0x279a0199310>,
 'Dam_tanki': <ee.featurecollection.FeatureCollection at 0x279a019a6d0>,
 'Salina': <ee.featurecollection.FeatureCollection at 0x279a01c24d0>,
 'Wetland_other': <ee.featurecollection.FeatureCollection at 0x2799f857650>,
 'Cactus_dominated': <ee.featurecollection.FeatureCollection at 0x279a0158c90>,
 'Deciduous_dominated': <ee.featurecollection.FeatureCollection at 0x2799e178190>,
 'Shrubland': <ee.featurecollection.FeatureCollection at 0x2799dc65190>,
 'Quarry': <ee.featurecollection.FeatureCollection at 0x279a01c00d0>,
 'Degraded_land': <ee.featurecollection.FeatureCollection at 0x279a019a2d0>,
 'Sand_coast': <ee.featurecollection.FeatureCollection at 0x2799fd8e750>,
 'Rock_coast': <ee.featurecollection.FeatureCollection at 0x279a01c1c50>,
 'Rock_formation': <ee.featurecollection.FeatureCollection at 0x279a01c0c50>,
 'Dead_mangrove': <e

In [75]:
# Run Checks
i = 0
j = 0

# # Every feature contains landcover property in the correct way
# for subclass in subClassNames:
#     for feature in subClassPoints[subclass].getInfo()['features']:
#         if 'landcover' not in feature['properties'].keys():
#             print(i, feature)
#             i += 1 
#         elif feature['properties']['landcover'] != subClassDict[subclass]:
#             print(j, feature['properties']['landcover'], subClassDict[subclass])
#             j += 1

# Expected length for each of the feature collections
for subclass in subClassNames:
    print(f'{subclass} {subClassPoints[subclass].size().getInfo()}')


Permanent_water 40
Mangrove 40
Dam_tanki 40
Salina 40
Wetland_other 40
Cactus_dominated 40
Deciduous_dominated 40
Shrubland 40
Quarry 40
Degraded_land 40
Sand_coast 40
Rock_coast 40
Rock_formation 40
Dead_mangrove 40
Road 40
Building 40
Sport_field 40


In [76]:
# # Pruning and Splitting in Training and groundTruth
trainingGroundTruth = {}
for subclass in subClassNames:
#     # Allow to be done randomly multiple times and then take the average?
#     # Return a dictionary containing subclass: {training: [], groundTruth: []}
    training, groundTruth = popRandomPoints(subClassPoints[subclass], 20)
    trainingGroundTruth[subclass] = {'training': training, 'groundTruth': groundTruth}

{name: {key: featureCol.getInfo() for key, featureCol in subDict.items()} for name, subDict in trainingGroundTruth.items()}

{'Permanent_water': {'training': {'type': 'FeatureCollection',
   'columns': {'landcover': 'Integer', 'system:index': 'String'},
   'features': [{'type': 'Feature',
     'geometry': {'type': 'Point',
      'coordinates': [-70.05132232873538, 12.569686344409568]},
     'id': '2_0000000000000000000b',
     'properties': {'landcover': 0}},
    {'type': 'Feature',
     'geometry': {'type': 'Point',
      'coordinates': [-70.0192767615108, 12.483490383546572]},
     'id': '2_00000000000000000002',
     'properties': {'landcover': 0}},
    {'type': 'Feature',
     'geometry': {'type': 'Point',
      'coordinates': [-69.96817268949235, 12.549914730189974]},
     'id': '2_0000000000000000001b',
     'properties': {'landcover': 0}},
    {'type': 'Feature',
     'geometry': {'type': 'Point',
      'coordinates': [-69.96335155418117, 12.54680726350093]},
     'id': '2_0000000000000000001e',
     'properties': {'landcover': 0}},
    {'type': 'Feature',
     'geometry': {'type': 'Point',
      'coo

In [3]:
# Create training and groundTruth dictionary of WorldCover classes

import math
import random
subClassPoints = eval(open('./output/subClassPoints.txt', 'r').read())

def popRandomPoints(collection, amt=1):
    """
    Remove amt number of points from collection, 
    return [Removed Points, old Points]
    where both are ee.FeatureCollections([])
    """
    if amt > 0 and amt <= len(collection['features']): 
        remaining_points = collection['features'].copy()
        removed_points = random.sample(remaining_points, amt)
        for point in removed_points:
            remaining_points.remove(point)
        return [removed_points, remaining_points]
    else:
        return [[], collection['features']]

classPoints = {}
for mainClassName, subClassLandCovers in classesFromSubClasses.items():
    landcover = classDict[mainClassName]
    nbSubLandcover = len(subClassLandCovers)
    # Take a fixed number of points per subclass, to ensure they are all represented
    points_per_landcover = math.ceil(20/nbSubLandcover)
    newCollection = {'training': {'features': []},
                     'groundTruth': {'features': []}}
    for subLandcover in subClassLandCovers:

        # Obtain subclassName
        subclassName = reverse_subClassDict[subLandcover]

        # Get the required points
        randomPointTraining, _ = popRandomPoints(subClassPoints[subclassName]['training'], points_per_landcover)
        randomPointsValidation , _ = popRandomPoints(subClassPoints[subclassName]['groundTruth'], points_per_landcover)

        # Store in new variable
        newCollection['training']['features'] += randomPointTraining
        newCollection['groundTruth']['features'] += randomPointsValidation

    # Remove the few that were selected in excess
    _, newCollection['training']  = popRandomPoints(newCollection['training'], len(newCollection['training']['features']) - 20)
    _, newCollection['groundTruth']  = popRandomPoints(newCollection['groundTruth'], len(newCollection['groundTruth']['features']) - 20)
    
    # Set landcover value to non subclass value
    newCollection['training'] = ee.FeatureCollection(newCollection['training']).map(lambda feature: feature.set({'landcover': landcover }))
    newCollection['groundTruth'] = ee.FeatureCollection(newCollection['groundTruth']).map(lambda feature: feature.set({'landcover': landcover }))

    classPoints[mainClassName] = newCollection

{name: {key: featureCol.getInfo() for key, featureCol in subDict.items()} for name, subDict in classPoints.items()}

{'Permanent_water': {'training': {'type': 'FeatureCollection',
   'columns': {'landcover': 'Integer', 'system:index': 'String'},
   'features': [{'type': 'Feature',
     'geometry': {'type': 'Point',
      'coordinates': [-69.95938188483913, 12.54488029434674]},
     'id': '2_00000000000000000021',
     'properties': {'landcover': 0}},
    {'type': 'Feature',
     'geometry': {'type': 'Point',
      'coordinates': [-69.95621863951223, 12.436817957700612]},
     'id': '2_00000000000000000027',
     'properties': {'landcover': 0}},
    {'type': 'Feature',
     'geometry': {'type': 'Point',
      'coordinates': [-69.98026423528248, 12.563874235368674]},
     'id': '2_00000000000000000017',
     'properties': {'landcover': 0}},
    {'type': 'Feature',
     'geometry': {'type': 'Point',
      'coordinates': [-70.06771851322651, 12.616468491166643]},
     'id': '2_0000000000000000000f',
     'properties': {'landcover': 0}},
    {'type': 'Feature',
     'geometry': {'type': 'Point',
      'co

In [ ]:
# Testing and showing structure of final dictionaries
print(subClassPoints['Permanent_water']['training'])
print(classPoints['Permanent_water']['groundTruth'])

for key, value in subClassPoints.items():
    print(key, len(value['training']['features']))
    print(key, len(value['groundTruth']['features']))

for key, value in classPoints.items():
    print(key, value['training']['features'])
    print(key, len(value['groundTruth']['features']))